# This notebook contains the analysis of the MC information truth of Tl source

In [ ]:
import os
import sys
import random
import tables as tb
import numpy  as np

import matplotlib as mpl
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt

from   invisible_cities.database               import load_db

import invisible_cities.reco.paolina_functions as plf
from   invisible_cities.reco.dst_functions     import load_xy_corrections

from   invisible_cities.io.dst_io              import load_dst
from   invisible_cities.io.hits_io             import load_hits
from   invisible_cities.io.hits_io             import load_hits_skipping_NN
from invisible_cities.io.mchits_io             import load_mchits

from   invisible_cities.types.ic_types         import xy
from   invisible_cities.types.ic_types         import NN

from   invisible_cities.evm.event_model        import Cluster, Hit

import invisible_cities.core.fit_functions     as fitf

from   invisible_cities.icaro.hst_functions    import hist
from   invisible_cities.icaro.hst_functions    import gausstext

from   invisible_cities.database               import load_db

In [ ]:
def merge_NN_hits(hits_all,hits_nonNN):

    # Iterate through the nonNN dictionary and update the energies including the NN hits from the "all" dictionary.
    for (evt,hc) in hits_nonNN.items():
        # Get the corresponding collection of all hits.
        hc_all = hits_all[evt]            
        # Add energy from all NN hits to hits in closest slice.
        for h1 in hc_all.hits:
            if(h1.Q == NN):
                # Find the hits to which the energy will be added.
                zdist_min = -1
                h_add = []
                for h2 in hc.hits:
                    zdist = np.abs(h1.Z - h2.Z)
                    if(zdist_min < 0 or zdist < zdist_min):
                        zdist_min = zdist
                        h_add = []
                        h_add.append(h2)
                    elif(zdist == zdist_min):
                        h_add.append(h2)

                # Add the energy.
                hadd_etot = sum([ha.E for ha in h_add])
                for ha in h_add:
                    ha.energy += h1.E*(ha.E/hadd_etot)

## Parameters

In [ ]:
DetGeo = load_db.DetectorGeo()
fwhmQbb = 0.006
vox_size = np.array([15,15,15],dtype=np.int16)    # voxel size
blob_radius = 21.   # blob radius in mm
file_base_name = 'tl_pmaps_7bar_NEXT_v1_00_05_v0.9.2_20171011_krmc_tl'
directory_name = '/home/Tl_v0.9.2'
start = 0 ## analyzes first file 0
numb = 5 ## number of files to be analyzed

### Containers for variables to be saved to file

In [ ]:
evt_energies = []
trk_energies = []
number_of_tracks = []
number_of_voxels = []
number_of_voxels_main = []
length_main = []
energy_main = []
l_eblob1 = []; l_eblob2 = []

### Loop on the files